In [ ]:

import pandas as pd
import numpy as np
from scipy import stats

import ipywidgets as widgets
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import missingno as msno
import warnings
warnings.filterwarnings('ignore')
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
from sklearn.model_selection import train_test_split, KFold, GridSearchCV


from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet,Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df=pd.read_csv('/content/Crop_production.csv')

In [ ]:
df1 = df.copy()

In [ ]:
df1.head(
)

In [ ]:
df1.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df1.drop('State_Name',axis=1,inplace=True)

In [ ]:
df1.drop('Crop_Type',axis=1,inplace=True)

In [ ]:
df1.drop('Crop',axis=1,inplace=True)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
state_encoder = LabelEncoder()
crop_type_encoder = LabelEncoder()
crop_encoder = LabelEncoder()

# Fit the encoders on the training data to avoid data leakage
state_encoder.fit(df['State_Name'])
crop_type_encoder.fit(df['Crop_Type'])
crop_encoder.fit(df['Crop'])

# Transform the categorical features in the entire DataFrame
df1['State_Name_Encoded'] = state_encoder.transform(df['State_Name'])
df1['Crop_Type_Encoded'] = crop_type_encoder.transform(df['Crop_Type'])
df1['Crop_Encoded'] = crop_encoder.transform(df['Crop'])

# Now, 'df' contains the encoded features and you can use it for training/prediction

In [ ]:
df1.info(

)

In [ ]:
X = df1.drop('Yield_ton_per_hec', axis=1)  # Features
y = df1['Yield_ton_per_hec']  # Target variable

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:

y_train = y_train.replace([np.inf, -np.inf], np.nan)

# Option 1: Remove rows with NaN values
# X_train = X_train[y_train.notna()]
# y_train = y_train[y_train.notna()]

# Option 2: Impute NaN values with the mean or median
y_train = y_train.fillna(y_train.mean())  # Or use y_train.median()

models = [RandomForestRegressor(), LinearRegression(), ElasticNet(), KNeighborsRegressor(), xgb.XGBRegressor(), Ridge()]
scores = dict()

for model in models:  # Changed 'models' to 'model' in the loop
    # Replace infinite or overly large values in X_train with NaN
    X_train = X_train.replace([np.inf, -np.inf], np.nan)
    # Impute NaN values with the mean
    X_train = X_train.fillna(X_train.mean())

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(y_pred)

    print(f'model: {str(model)}')
    print(f'RMSE: {mean_squared_error(y_test, y_pred)}')
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print('-' * 30, '\n')

In [ ]:
import xgboost as xgb

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
df1.info()

In [ ]:
# Define input widgets for each feature
N_input = widgets.IntText(description="N:")
P_input = widgets.IntText(description="P:")
K_input = widgets.IntText(description="K:")
temperature_input = widgets.FloatText(description="Temperature:")
pH_input = widgets.FloatText(description="pH:")
area_input = widgets.IntText(description="Area (hectares):")
rainfall_input = widgets.FloatText(description="Rainfall:")
# Assuming 'Production_in_tons' is a necessary feature, include it as well:
production_input = widgets.FloatText(description="Production (tons):") # Placeholder for now, consider its actual purpose

# Dropdown widgets for categorical features
state_input = widgets.Dropdown(
    options=df['State_Name'].unique().tolist(),
    description="State:"
)
crop_type_input = widgets.Dropdown(
    options=df['Crop_Type'].unique().tolist(),
    description="Crop Type:"
)
crop_input = widgets.Dropdown(
    options=df['Crop'].unique().tolist(),
    description="Crop:"
)

# Function to handle prediction when button is clicked
def predict_on_input(b):
    input_data = pd.DataFrame({
        'N': [N_input.value],
        'P': [P_input.value],
        'K': [K_input.value],
        'temperature': [temperature_input.value],
        'pH': [pH_input.value],
        'Area_in_hectares': [area_input.value],
        'rainfall': [rainfall_input.value],
        'Production_in_tons': [production_input.value],
        'State_Name': [state_input.value],  # Get selected state
        'Crop_Type': [crop_type_input.value],  # Get selected crop type
        'Crop': [crop_input.value]  # Get selected crop
    })

    # Encode categorical features
    input_data['State_Name_Encoded'] = state_encoder.transform(input_data['State_Name'])
    input_data['Crop_Type_Encoded'] = crop_type_encoder.transform(input_data['Crop_Type'])
    input_data['Crop_Encoded'] = crop_encoder.transform(input_data['Crop'])

    # Ensure the order of columns matches the training data
    input_data = input_data[['N', 'P', 'K', 'pH', 'rainfall', 'temperature', 'Area_in_hectares', 'Production_in_tons',
                            'State_Name_Encoded', 'Crop_Type_Encoded', 'Crop_Encoded']]  # Added encoded features

    predictions = model.predict(input_data)
    predicted_value = predictions[0]*907.185
    print("Predicted Value in tons:", predictions[0])


# Create a button to trigger prediction
predict_button = widgets.Button(description="Predict")
predict_button.on_click(predict_on_input)

# Display the input widgets and button
display(N_input, P_input, K_input, temperature_input, pH_input, area_input,
        rainfall_input, production_input, state_input, crop_type_input, crop_input, predict_button)